# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,swidnik,51.2190,22.6962,22.61,55,20,4.12,PL,1720085790
1,1,utrik,11.2278,169.8474,27.91,81,76,6.95,MH,1720085793
2,2,north battleford,52.7834,-108.2847,15.78,100,0,3.09,CA,1720085794
3,3,bredasdorp,-34.5322,20.0403,15.62,59,4,4.77,ZA,1720085795
4,4,sorong,-0.8833,131.2500,26.95,79,100,1.04,ID,1720085764


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)


# Display the map
# YOUR CODE HERE
map_plot_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

ideal_temp_cond_city = ((city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27))
ideal_wind_cond_city = (city_data_df["Wind Speed"] < 4.5)
ideal_cloud_cond_city = (city_data_df["Cloudiness"] == 0)

city_data_ideal_cond = city_data_df.loc[(ideal_temp_cond_city & ideal_wind_cond_city & ideal_cloud_cond_city ), :]

# Drop any rows with null values
# YOUR CODE HERE

# Drop all rows with missing information
city_data_ideal_cond = city_data_ideal_cond.dropna()

# Display sample data
# YOUR CODE HERE
city_data_ideal_cond

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,howard springs,-12.4970,131.0470,26.95,32,0,0.71,AU,1720085874
65,65,stephenville,32.2207,-98.2023,26.80,80,0,3.09,US,1720085882
167,167,newman,37.3138,-121.0208,26.13,30,0,1.68,US,1720086022
242,242,laguna,38.4210,-121.4238,25.98,54,0,1.54,US,1720086122
299,299,salida,37.7058,-121.0849,26.74,45,0,1.54,US,1720086196
332,332,feraoun,36.5604,4.8545,21.66,20,0,1.20,DZ,1720086248
418,418,kasane,-17.8167,25.1500,26.05,22,0,3.09,BW,1720086349
443,443,brandon,27.9378,-82.2859,26.61,93,0,0.45,US,1720086400
450,450,nova vicosa,-17.8919,-39.3719,21.40,77,0,2.31,BR,1720086409


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_ideal_cond.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity", "Hotel Name"]]
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
59,howard springs,AU,-12.4970,131.0470,32,
65,stephenville,US,32.2207,-98.2023,80,
167,newman,US,37.3138,-121.0208,30,
242,laguna,US,38.4210,-121.4238,54,
299,salida,US,37.7058,-121.0849,45,
332,feraoun,DZ,36.5604,4.8545,20,
418,kasane,BW,-17.8167,25.1500,22,
443,brandon,US,27.9378,-82.2859,93,
450,nova vicosa,BR,-17.8919,-39.3719,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bijelo polje - nearest hotel: Hotel Franca
shiprock - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
hamilton - nearest hotel: North Vista Manor
waycross - nearest hotel: Holiday Inn Express & Suites
jamestown - nearest hotel: DoubleTree Jamestown
bonne terre - nearest hotel: Bonneville Motel
laguna - nearest hotel: Holiday Inn Express & Suites
cristuru secuiesc - nearest hotel: Kúria Vendégház
badger - nearest hotel: Clarion Hotel & Suites
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
koryazhma - nearest hotel: Виконда
sparti - nearest hotel: Maniatis
revda - nearest hotel: Металлург
mermoz boabab - nearest hotel: La maison blanche
mount pleasant - nearest hotel: Old Village Post House
nova vicosa - nearest hotel: Pousada Pontal da Barra
rio de janeiro - nearest hotel: Ibis Rio porto atlantico
qorveh - nearest hotel: No hotel found
vaison-la-romaine - nearest hotel: Hostellerie le Beffro

,City,Country,Lat,Lng,Humidity,Hotel Name
13,bijelo polje,ME,43.0337,19.7470,51,Hotel Franca
39,shiprock,US,36.7855,-108.6870,14,No hotel found
58,port elizabeth,ZA,-33.9180,25.5701,60,Waterford Hotel
82,hamilton,US,39.1834,-84.5333,82,North Vista Manor
109,waycross,US,31.2136,-82.3540,94,Holiday Inn Express & Suites
125,jamestown,US,42.0970,-79.2353,88,DoubleTree Jamestown
193,bonne terre,US,37.9231,-90.5554,89,Bonneville Motel
202,laguna,US,38.4210,-121.4238,53,Holiday Inn Express & Suites
215,cristuru secuiesc,RO,46.2833,25.0333,60,Kúria Vendégház
219,badger,US,64.8000,-147.5333,42,Clarion Hotel & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)